In [23]:
import pandas as pd
import os
import collections

In [24]:
# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
output_vision_file = os.path.join(relative_path, 'Vision2050_longlist.csv')
taz_reg_geog = '../../../scripts/summarize/inputs/TAZ_Reg_Geog.csv'

In [25]:
# Load network results
network_df = pd.read_excel(os.path.join(relative_path,'network/') + r'network_summary_detailed.xlsx',
                  sheetname='Network Summary')

In [26]:
# System performance measures
# VMT
network_df['all_facility_vmt'] = network_df['arterial_vmt']+ network_df['connectors_vmt']+ network_df['highway_vmt']
total_vmt = network_df['all_facility_vmt'].sum()
output_dict = {}
output_dict = collections.OrderedDict(output_dict)
output_dict[('Daily VMT','Regional', 'Total')] = total_vmt

In [27]:
#VHT
network_df['all_facility_vht'] = network_df['arterial_vht']+ network_df['connectors_vht']+ network_df['highway_vht']
total_vht = network_df['all_facility_vht'].sum()
output_dict[('Daily VHT', 'Regional', 'Total')] = total_vht

In [28]:
# Delay
network_df['all_facility_delay'] = network_df['arterial_delay']+ network_df['connectors_delay']+ network_df['highway_delay']
total_delay = network_df['all_facility_delay'].sum()
output_dict[('Daily Delay', 'Regional', 'Total')] = total_delay

In [29]:
# Total Delay Hours Daily by Facility Type
df_fac = pd.DataFrame(network_df.sum()[['arterial_delay','highway_delay']])
df_fac = df_fac.reset_index()
df_fac.columns = ['Facility Type', 'Delay']
#df_fac.index = df_fac['Facility Type']
#df_fac.drop('Facility Type', axis=1, inplace=True)
df_fac.loc['Total'] = df_fac.sum()
output_dict[('Daily Arterial Delay Hours', 'Regional', 'Total')] = df_fac['Delay'].loc[df_fac['Facility Type'] == 'arterial_delay'].values[0]
output_dict[('Daily Highway Delay Hours', 'Regional', 'Total')] = df_fac['Delay'].loc[df_fac['Facility Type'] == 'highway_delay'].values[0]

In [30]:
# Daily Transit Boardings
transit_df = pd.read_excel(r'../../../outputs/network/network_summary_detailed.xlsx', sheetname='Transit Summaries')
tod_list = ['5to6','6to7','7to8','8to9','9to10','10to14','14to15','15to16','16to17','17to18','18to20']
transit_df = transit_df[[tod+'_board' for tod in tod_list]+['route_code']]
transit_df = transit_df.fillna(0)
transit_df['line_total'] = transit_df[[tod+'_board' for tod in tod_list]].sum(axis=1)
output_dict[('Daily Transit Boardings', 'Regional', 'Total')]=transit_df['line_total'].sum()


In [31]:
mode_dict = {0: 'Other', 1:'Walk',2:'Bike',3:'SOV',4:'HOV2',5:'HOV3+',6:'Transit',8:'School Bus'}
annualization_factor = 300

In [32]:
# Load Daysim results and join to geography
trip = pd.read_csv(os.path.join(relative_path,r'daysim/') + r'_trip.tsv', sep='\t')
print trip.shape
person = pd.read_csv(os.path.join(relative_path,r'daysim/') + r'_person.tsv', sep='\t')
hh = pd.read_csv(os.path.join(relative_path,r'daysim/') + r'_household.tsv', sep='\t')
taz_geog = pd.read_csv(taz_reg_geog)

(22394210, 28)


In [33]:
person =pd.merge(person[['hhno','pno', 'psexpfac']],hh[['hhno','hhtaz']], on = 'hhno', suffixes=['','_x'])
person = pd.merge(person, taz_geog, left_on = 'hhtaz', right_on = 'taz_p')
#del hh

trip = pd.merge(trip[['hhno', 'pno', 'opurp', 'dpurp', 'travdist', 'travtime', 'mode', 'dorp', 'dtaz']], person, on = ['hhno', 'pno'], suffixes=['','_x'], how = 'inner' )
trip = pd.merge(trip, taz_geog, left_on = 'dtaz', right_on ='taz_p', suffixes = ['', '_tripdest'])

In [34]:
# Average HBW/ Non-HBW Trip Distance
trip['Trip Type'] = 'Not Home-Based Work'
trip.ix[(((trip['opurp']==0) & (trip['dpurp']==1)) | ((trip['opurp']==1) & (trip['dpurp']==0))),'Trip Type']= 'Home-Based Work'
hbw_trips = trip.loc[trip['Trip Type']=='Home-Based Work']
nhbw_trips = trip.loc[trip['Trip Type']=='Not Home-Based Work']
output_dict[('Commute Trip Length', 'Regional', 'Total')]=hbw_trips['travdist'].mean()
output_dict[('Other Trip Length', 'Regional', 'Total')]= nhbw_trips['travdist'].mean()

In [35]:
# Average Trip Distance by Geography/Commute/Non-Commute
hbw_geog=hbw_trips.groupby('rg_proposed').mean()['travdist'].reset_index()
for index, row in hbw_geog.iterrows():
    output_dict[('Commute Trip Length', row['rg_proposed'], 'Total')] = row['travdist']
    
nhbw_geog=nhbw_trips.groupby('rg_proposed').mean()['travdist'].reset_index()
for index, row in nhbw_geog.iterrows():
    output_dict[('Other Trip Length', row['rg_proposed'], 'Total')] = row['travdist']

In [36]:
# All Trip Mode Share
mode_df = trip.groupby('mode').count()['hhno'].reset_index()
mode_df['Mode Share']= mode_df['hhno']/mode_df.sum()['hhno']
mode_df.replace({'mode':mode_dict}, inplace=True)

for index, row in mode_df.iterrows():
    output_dict[('All Trip Mode Share', 'Regional', row['mode'])]=row['Mode Share']

mode_df_geog = trip.groupby(['mode', 'rg_proposed_tripdest']).count()['hhno'].reset_index()
mode_df_geog['Mode Share'] = mode_df_geog['hhno']/mode_df_geog.groupby('rg_proposed_tripdest').sum()['hhno']
mode_df_geog.replace({'mode':mode_dict}, inplace=True)


for index, row in mode_df_geog.iterrows():
    output_dict[('All Trip Mode Share', row['rg_proposed_tripdest'], row['mode'])]=row['Mode Share']


In [39]:
mode_df_geog.groupby('rg_proposed_tripdest').sum()['hhno']

rg_proposed_tripdest
CitiesTowns    1779954
Core           6034929
HCT            3544962
Metro          8250902
Rural          1632964
UU             1046961
Name: hhno, dtype: int64

In [15]:
# Commute Trip Mode Share
mode_df = hbw_trips.groupby('mode').count()['hhno'].reset_index()
mode_df['Mode Share']= mode_df['hhno']/mode_df.sum()['hhno']
mode_df.replace({'mode':mode_dict}, inplace=True)

for index, row in mode_df.iterrows():
    output_dict[('Commute Trip Mode Share', 'Regional', row['mode'])]=row['Mode Share']

mode_df_geog = hbw_trips.groupby(['rg_proposed_tripdest', 'mode']).count()['hhno'].reset_index()
mode_df_geog['Mode Share'] = mode_df_geog['hhno']/mode_df_geog.groupby('rg_proposed_tripdest').sum()['hhno']
mode_df_geog.replace({'mode':mode_dict}, inplace=True)


for index, row in mode_df_geog.iterrows():
    output_dict[('Commute Trip Mode Share', row['rg_proposed_tripdest'], row['mode'])]=row['Mode Share']

In [16]:
# Non-Commute Trip Mode Share
mode_df = nhbw_trips.groupby('mode').count()['hhno'].reset_index()
mode_df['Mode Share']= mode_df['hhno']/mode_df.sum()['hhno']
mode_df.replace({'mode':mode_dict}, inplace=True)

for index, row in mode_df.iterrows():
    output_dict[('Non-Commute Trip Mode Share', 'Regional', row['mode'])]=row['Mode Share']

mode_df_geog =nhbw_trips.groupby(['rg_proposed_tripdest', 'mode']).count()['hhno'].reset_index()
mode_df_geog['Mode Share'] = mode_df_geog['hhno']/mode_df_geog.groupby('rg_proposed_tripdest').sum()['hhno']
mode_df_geog.replace({'mode':mode_dict}, inplace=True)


for index, row in mode_df_geog.iterrows():
    output_dict[('Non-Commute Trip Mode Share', row['rg_proposed_tripdest'], row['mode'])]=row['Mode Share']

In [17]:
# VMT per resident per day
driver_trips = trip[['rg_proposed','travdist']].loc[trip['dorp']==1]
output_dict[('Average VMT per Resident', 'Regional', 'Total')]=driver_trips['travdist'].sum()/ person['psexpfac'].sum()

driver_trips_geog = pd.DataFrame(driver_trips.groupby('rg_proposed').sum()['travdist']/person.groupby('rg_proposed').sum()['psexpfac'])
driver_trips_geog.reset_index(inplace=True)
driver_trips_geog.columns = ['rg_proposed','vmt']

for index, row in driver_trips_geog.iterrows():
    output_dict[('Average VMT per Resident',row['rg_proposed'], 'Total')]=row['vmt']

In [18]:
# Number and Percent of People Walking, Biking, or Transiting
bike_walk_t_trips = trip[trip['mode'].isin([1,2,6])]

df = bike_walk_t_trips.groupby(['hhno','pno']).count()
df = df.reset_index()
df = df[['hhno','pno']]
df['bike_walk_t'] = 1

df['bike_walk_t'] = df['bike_walk_t'].fillna(0)


output_dict[('Share of People Walking, Biking, and Using Transit', 'Regional', 'Total')]=float(df['bike_walk_t'].sum())/person['psexpfac'].sum()
output_dict[('Total People Walking, Biking, and Using Transit', 'Regional', 'Total')]=df['bike_walk_t'].sum()

In [19]:
df = bike_walk_t_trips.groupby(['hhno','pno', 'rg_proposed']).count()
df = df.reset_index()
df = df[['hhno','pno','rg_proposed']]
df['bike_walk_t'] = 1

df['bike_walk_t'] = df['bike_walk_t'].fillna(0)


df_geog_share=pd.DataFrame(df.groupby('rg_proposed').sum()['bike_walk_t']/person.groupby('rg_proposed').sum()['psexpfac'])
df_geog_total=pd.DataFrame(df.groupby('rg_proposed').sum()['bike_walk_t'])
df_geog_share.reset_index(inplace=True)
df_geog_total.reset_index(inplace=True)

df_geog_share.columns = ['rg_proposed', 'wbt']
df_geog_total.columns = ['rg_proposed', 'wbt']

for index, row in df_geog_share.iterrows():
    output_dict[('Share of Residents Walking, Biking,or Using Transit',row['rg_proposed'], 'Share')]=row['wbt']
for index, row in df_geog_total.iterrows():
    output_dict[('Total of Residents Walking, Biking,or Using Transit',row['rg_proposed'], 'Total')]=row['wbt']

In [20]:
output_df = pd.DataFrame(output_dict.keys(), index = output_dict.values()).reset_index()
output_df.columns = ['Value', 'Data Item', 'Geography', 'Grouping']

In [21]:
output_df=output_df[output_df['Grouping']!='Other']
output_df = output_df.sort_values(by = ['Data Item', 'Geography', 'Grouping'])
output_df.to_csv(output_vision_file)

In [22]:
output_df

,Value,Data Item,Geography,Grouping
0,1.049290e+08,Daily VMT,Regional,Total
1,3.490024e+06,Daily VHT,Regional,Total
2,8.805924e+05,Daily Delay,Regional,Total
3,4.330399e+05,Daily Arterial Delay Hours,Regional,Total
4,4.475525e+05,Daily Highway Delay Hours,Regional,Total
5,1.515906e+06,Daily Transit Boardings,Regional,Total
6,1.039140e+01,Commute Trip Length,Regional,Total
7,4.165995e+00,Other Trip Length,Regional,Total
8,1.504272e+01,Commute Trip Length,CitiesTowns,Total
9,9.747337e+00,Commute Trip Length,Core,Total
